## Board game recommendation engine
### Example of user rating predictor for Matt Borthwick's rating challenge
#### Using Single Value Decomposition, and Alternating Least Squares

#### John Burt


#### Purpose of this notebook:

This is an example of how to implement a user rating predictor, using the boardgamegeek dataset. The method reads user data and the the provided test data with missing ratings, creates a ratings predictor based on the user data, generates ratings predictions for the test data and saves the results to a csv file that can be presented to Matt's website for evaluation.

Method transforms the user ratings data via Single Value Decomposition (SVD) into a 4D "user-space", where users with similar ratings are closer in the space. For a given test set of userID and gameID, the predictor:
- Finds the user-space coordinate of the user.
- Searches for nearby users who have rated the game.
- Returns the mean rating of a specified number of neighbors who have rated the game.

Run with the all users data, this method gets an RMSE score of 1.233

**Note about Alternating Least Squares:** Prior to the SVD transform to user-space, the raw ratings data is converted to a 2D array of userID x gameID, with cells containing either ratings, or NaNs for games that a user hasn't rated (which is most of the array). SVD wants a fully filled (dense) array, so the unrated cells with NaNs need to be filled with something useful. There are several ways to do this, including just setting the NaN cells to 0, or using an imputer to fill the empty cells with mean game ratings. Using those methods had poor results, so I tried another method called Alternating Least Squares (ALS), which improves the accuracy by quite a bit. The downside to ALS is that it is very memory and processor intensive: you cannot run ALS on the full dataset due to memory limitations, and even if you restrict the training set to only users in the test set, as I do here, it can take a long time for the ALS to finish. So, fair warning!

In [41]:
# set up the notebook and load the data

# remove warnings
import warnings
warnings.filterwarnings('ignore')
# ---

%matplotlib inline
import pandas as pd
pd.options.display.max_columns = 100
from matplotlib import pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')
import numpy as np

from datetime import datetime

pd.options.display.max_rows = 100

# load the boardgame user data
testdata = pd.read_csv('boardgame-users-test.csv') 
#userdata = pd.read_csv('boardgame-elite-users.csv')
#userdata = pd.read_csv('boardgame-frequent-users.csv')
userdata = pd.read_csv('boardgame-users.csv') 

# rename the userID column
userdata=userdata.rename(columns = {"Compiled from boardgamegeek.com by Matt Borthwick":'userID'})

# load the boardgame title data
titledata = pd.read_csv('boardgame-titles.csv')

# rename the gameID column
titledata=titledata.rename(columns = {"boardgamegeek.com game ID":'gameID'})

# for titledata set game ID as the index
titledata = titledata.set_index("gameID")

Weighted Alternating Least Squares algorithm. I found this at the source below. It seems popular as a method of inferring missing ratings to fill ratings matrices prior to feeding them into PCA or SVD analyses. For more on this method, see the link below.

I got the idea to use this and adapted the code from:
https://bugra.github.io/work/notes/2014-04-19/alternating-least-squares-method-for-collaborative-filtering/

### WARNING: this function is computationally heavy. It can take a while to complete and if the dataset is too large, the code will barf out with a memory error.

In [42]:
def get_error(Q, X, Y, W):
    return np.sum((W * (Q - np.dot(X, Y)))**2)

def do_ALS(Q, n_iterations=10, lambda_=0.1, n_factors=100, weighted=True, verbose=True ):
    
    W = Q>0.5
    W[W == True] = 1
    W[W == False] = 0
    # To be consistent with our Q matrix
    W = W.astype(np.float64, copy=False)

    m, n = Q.shape

    X = 5 * np.random.rand(m, n_factors) 
    Y = 5 * np.random.rand(n_factors, n)
    
    if verbose: print("\n")

    if not weighted:
        errors = []
        for ii in range(n_iterations):
            X = np.linalg.solve(np.dot(Y, Y.T) + lambda_ * np.eye(n_factors), 
                                np.dot(Y, Q.T)).T
            Y = np.linalg.solve(np.dot(X.T, X) + lambda_ * np.eye(n_factors),
                                np.dot(X.T, Q))
            if ii % 2 == 0:
                if verbose: print("iteration %d, error = %4.2f"%(ii,get_error(Q, X, Y, W)))

            errors.append(get_error(Q, X, Y, W))
        Q_hat = np.dot(X, Y)
        if verbose: print("Error of rated games: %4.2f"%(get_error(Q, X, Y, W)))

    else:
        errors = []
        for ii in range(n_iterations):
            for u, Wu in enumerate(W):
                X[u] = np.linalg.solve(np.dot(Y, np.dot(np.diag(Wu), Y.T)) + lambda_ * np.eye(n_factors),
                                       np.dot(Y, np.dot(np.diag(Wu), Q[u].T))).T
            for i, Wi in enumerate(W.T):
                Y[:,i] = np.linalg.solve(np.dot(X.T, np.dot(np.diag(Wi), X)) + lambda_ * np.eye(n_factors),
                                         np.dot(X.T, np.dot(np.diag(Wi), Q[:, i])))
            if ii % 2 == 0:
                if verbose: print("iteration %d, error = %4.2f"%(ii,get_error(Q, X, Y, W)))
            errors.append(get_error(Q, X, Y, W))

        if verbose: print("iteration %d, error = %4.2f"%(ii,get_error(Q, X, Y, W)))
        Q_hat = np.dot(X,Y)

    return Q_hat, X, Y, errors

For a given test set of userID and gameID, search user-space for nearby users who have rated the game. Return a mean rating for a specified number of them.

In [43]:
from scipy.spatial.distance import cdist

def estimate_ratings(testdata, coords, train_p, numneighbors2use):
    
    # iterate through all test pairs of userID, gameID
    rating = []
    for index, rec in testdata.iterrows():
        
        # index of test user in training data
        userindex = train_p.index.get_loc(rec.userID) 

        # coordinate in user-space of test user
        targetcoord = coords[userindex,:]

        # get euclidean distances of all points to targetcoord    
        dists = cdist(np.reshape(targetcoord,(1,-1)),coords) 

        # sort by distance
        ind, = np.argsort(dists)

        # Search neighbors for ratings for the specified game.
        # Start with nearest neighbor and collect specified number of 
        #  nearby ratings.
        ratings = []
        for user in train_p.index[ind]:
            if ~np.isnan(train_p[rec.gameID][user]):
                ratings.append(train_p[rec.gameID][user])
                if len(ratings) >= numneighbors2use:
                    break

        # get mean rating of collected user ratings
        rating.append(np.mean(ratings))

    return np.array(rating)

In [44]:
from sklearn.decomposition import PCA, SVD, SparsePCA, KernelPCA, TruncatedSVD, MiniBatchSparsePCA
from sklearn.preprocessing import normalize, StandardScaler, Normalizer

# only train with users in the test data set
traindata = userdata[userdata.userID.isin(set(testdata.userID))]

# only test with users that exist in both training and test set
testdata_filt = testdata[testdata.userID.isin(set(userdata.userID))]

# pivot the training data to get roaw=userID, cols=gameID, cells=ratings (or NaNs if no rating)
train_p = traindata.pivot(index="userID", columns="gameID", values="rating")

# run ALS on pivit data to fill in NaN cells with a useful ratings estimate
lambda_ = 0.1 # note: changing this doesn't seem to affect much
n_factors = 10 # smaller #factors seems to give better results
n_iterations = 8 # 8-10 iterations works best for all-user data, 15-20 for elite & frequent users data
# train_p_filled is our filled in matrix, errors lets us plot how things went
train_p_filled, X, Y, errors = do_ALS(train_p.fillna(0).values, n_iterations=n_iterations, 
                             lambda_=lambda_, n_factors=n_factors, weighted=True )

# convert numpy array output back to pandas dataframe
train_p_filled = pd.DataFrame(train_p_filled,index=train_p.index,columns=train_p.columns)

# threshold ratings to between 1-10
train_p_filled = train_p_filled.clip_upper(10)
train_p_filled = train_p_filled.clip_lower(1)

# transform filled pivot array into 4D "user-space" coordinates
numdims =  4
coords = TruncatedSVD(n_components=numdims).fit_transform(train_p_filled)

# get estimated ratings for test set
numneighbors2use = 50 # 50 users seems to work best
ratings_est = np.array(estimate_ratings(testdata_filt, coords, train_p_filled, numneighbors2use))




iteration 0, error = 1599950.24
iteration 2, error = 1322745.33
iteration 4, error = 1256091.93
iteration 6, error = 1236436.40
iteration 7, error = 1231704.39


 Generate submission file for Matt's website.
 
 Submit the file here: 
 http://dive-into.info/5848/ 

In [53]:
version = 2
initials = "JMB"
test_pred = testdata_filt.copy()
test_pred["rating"] = ratings_est
test_pred = test_pred.set_index(["userID"])
test_pred.to_csv(initials+"_predicted-v"+str(version)+".csv")
